In [ ]:
import ast
import pathlib

In [ ]:
REPO_ROOT = pathlib.Path('../..').resolve()
LIBRARY_PATH = REPO_ROOT / 'lib'

filepath = REPO_ROOT / 'lib' / 'pymedphys' / '_streamlit' / 'apps' / 'metersetmap' / 'main.py'
relative_path = filepath.relative_to(LIBRARY_PATH)

In [ ]:
CONVERSIONS = {
    "attr": "attrs",
    "PIL": "Pillow",
    "Image": "Pillow",
    "mpl_toolkits": "matplotlib",
    "dateutil": "python_dateutil",
    "skimage": "scikit-image",
    "yaml": "PyYAML",
    "plt": "matplotlib"
}

In [ ]:
with open(filepath, "r") as file:
    module_contents = file.read()
    
parsed = ast.parse(module_contents)
all_import_nodes = [
    node for node in ast.walk(parsed) 
    if isinstance(node, ast.Import) or isinstance(node, ast.ImportFrom)
]

import_nodes = [
    node for node in all_import_nodes 
    if isinstance(node, ast.Import)
]
import_from_nodes = [
    node for node in all_import_nodes 
    if isinstance(node, ast.ImportFrom)
]

In [ ]:
imports = set()

for node in import_nodes:
    for alias in node.names:
        imports.add(alias.name)
        
for node in import_from_nodes:
    if node.level == 0:
        if node.module.startswith('pymedphys._imports'):
            for alias in node.names:
                try:
                    name = CONVERSIONS[alias.name]
                except KeyError:
                    name = alias.name
                imports.add(name)
        else:
            for alias in node.names:
                imports.add(f"{node.module}.{alias.name}")
                
    else:
        module = '.'.join(relative_path.parts[:-node.level])
        for alias in node.names:
            imports.add(f"{module}.{alias.name}")

In [ ]:
imports